# 💾 Working with Data Files: JSON and CSV

**DS105W W03 NB02 – Data for Data Science (Winter Term 2025/2026)**

<div style="font-family: system-ui; padding: 20px 30px 20px 20px; background-color: #FFFFFF; border-left: 8px solid #ED9255; border-radius: 8px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1);max-width:600px;color:#212121;">

**Lecture Demonstration Notebook**

- 📅 Date: 05 February 2026
- 👤 Instructor: Dr Jon Cardoso-Silva
- 🎯 Purpose: Learn to save and read data files to avoid repeated API calls

<span style="display:block;line-height:1.15em;color:#666666;font-size:0.9em;">

🥅 **Learning Goals**

 i) Understand why saving data locally is important,
 ii) Learn to write and read JSON files,
 iii) Learn to write and read CSV files using Python's csv module,
 iv) Connect file operations to the data workflows you'll build in W04.

</span>

</div>

⚙️ **Importing libraries**

We'll need these for working with APIs and files.

In [2]:
import requests
import json
import csv

## Section 1: Collecting Data from an API (Recap)

Back in 🗣️ [**W02 Lecture**](https://lse-dsi.github.io/DS105/2025-2026/winter-term/weeks/week02/lecture.html), you learned how to collect data from APIs using `requests.get()`. Let's do a quick recap with a simple weather API call.

**Important:** Every time you run `requests.get()`, you're asking the Open-Meteo servers to send you data. If everyone in the class does this repeatedly, we'll hit rate limits and the API might block us. The solution is to save the data locally once, then work with the saved version.

In [3]:
# Request a small sample of weather data
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
    "latitude": 51.5074,
    "longitude": -0.1278,
    "start_date": "2025-07-01",
    "end_date": "2025-07-31",
    "daily": "temperature_2m_max",
    "timezone": "Europe/London"
}

response = requests.get(url, params=params)
weather_data = response.json()

print("Data collected successfully!")
print(f"We got data for {len(weather_data['daily']['time'])} days")
print(weather_data['daily'])

Data collected successfully!
We got data for 31 days
{'time': ['2025-07-01', '2025-07-02', '2025-07-03', '2025-07-04', '2025-07-05', '2025-07-06', '2025-07-07', '2025-07-08', '2025-07-09', '2025-07-10', '2025-07-11', '2025-07-12', '2025-07-13', '2025-07-14', '2025-07-15', '2025-07-16', '2025-07-17', '2025-07-18', '2025-07-19', '2025-07-20', '2025-07-21', '2025-07-22', '2025-07-23', '2025-07-24', '2025-07-25', '2025-07-26', '2025-07-27', '2025-07-28', '2025-07-29', '2025-07-30', '2025-07-31'], 'temperature_2m_max': [33.5, 23.0, 23.6, 26.1, 21.6, 22.5, 19.5, 21.9, 25.2, 30.0, 31.6, 28.8, 27.7, 24.9, 21.6, 23.8, 25.8, 29.4, 22.5, 20.4, 22.6, 22.3, 22.0, 20.8, 25.4, 23.2, 21.5, 24.1, 21.9, 24.7, 22.6]}


In [4]:
# What does this data look like?

**The Problem:** Every time you run the code above, you're downloading the same data again. This wastes time and risks hitting API rate limits.

**The Solution:** Save it to a file once, then read from the file when you need it.

## Section 2: Writing and Reading JSON Files

JSON (JavaScript Object Notation) is perfect for storing structured data like API responses. It looks just like Python dictionaries and lists, which makes it natural to work with.

### 2.1 Writing Data to a JSON File

Let's save the weather data we just collected so we don't need to request it again.

In [5]:
# Create a data folder if it doesn't exist (we'll do this via Terminal in the lab)
# For now, assume we have a data/ folder ready

# Write the weather data to a JSON file
with open('data/july_2025_weather.json', 'w') as f:
    json.dump(weather_data, f, indent=2)

print("Data saved to data/july_2025_weather.json")

Data saved to data/july_2025_weather.json


**What just happened?**

- `open('data/july_2025_weather.json', 'w')` opens a file for writing (`'w'` means write mode)
- `json.dump(weather_data, f, indent=2)` writes the dictionary to the file in JSON format
- `indent=2` makes the file human-readable with nice formatting
- The `with` statement automatically closes the file when done

### 2.2 Reading Data from a JSON File

Now we can read the saved data without making another API call.

In [6]:
# Read the data back from the file
with open('data/july_2025_weather.json', 'r') as f:
    loaded_data = json.load(f)

print("Data loaded from file!")
print(f"We have data for {len(loaded_data['daily']['time'])} days")

Data loaded from file!
We have data for 31 days


In [7]:
# Verify it's the same data

**Benefits of this approach:**

1. Faster - no waiting for network requests
2. Works offline - no internet required once saved
3. Respectful - doesn't overload the API servers
4. Reproducible - you're always working with the exact same data

## Section 3: Writing and Reading CSV Files

CSV (Comma-Separated Values) files are simpler than JSON but perfect for tabular data. You've already worked with CSV files in 📝 [**W01 Practice**](https://lse-dsi.github.io/DS105/2025-2026/winter-term/practice/week01.html) when loading them with pandas.

Now let's learn how to create CSV files using Python's built-in `csv` module (similar to what you saw in DataQuest this week).

### 3.1 Writing Data to a CSV File

Let's transform our weather data into a simple CSV file with dates and temperatures.

In [8]:
# Extract the data we need
dates = loaded_data['daily']['time']
temps = loaded_data['daily']['temperature_2m_max']

# Write to CSV file
with open('data/july_2025_temps.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    
    # Write header row
    writer.writerow(['date', 'max_temp_c'])
    
    # Write data rows
    for i in range(len(dates)):
        writer.writerow([dates[i], temps[i]])

print("CSV file created: data/july_2025_temps.csv")

CSV file created: data/july_2025_temps.csv


**What's happening here?**

- `csv.writer(f)` creates a CSV writer object
- `writer.writerow(['date', 'max_temp_c'])` writes the header
- The loop goes through each index position (0, 1, 2, ...)
- For each position `i`, we write the date and temperature at that position

### 3.2 Reading Data from a CSV File

Now let's read it back using the `csv` module.

In [20]:
# Read the CSV file
with open('data/july_2025_temps.csv', 'r') as f:
    reader = csv.reader(f)
    print(reader)
    print(next(reader))
    # Skip the header
    header = next(reader)
    print(f"Header: {header}")
    
    # Read the data
    rows = list(reader)
    print(rows)
    print(list(reader))
    print(f"\nFirst 5 rows:")
    for row in rows[:5]:
        print(f"  {row[0]}: {row[1]}°C")

['date', 'max_temp_c']
Header: ['2025-07-01', '33.5']
[['2025-07-02', '23.0'], ['2025-07-03', '23.6'], ['2025-07-04', '26.1'], ['2025-07-05', '21.6'], ['2025-07-06', '22.5'], ['2025-07-07', '19.5'], ['2025-07-08', '21.9'], ['2025-07-09', '25.2'], ['2025-07-10', '30.0'], ['2025-07-11', '31.6'], ['2025-07-12', '28.8'], ['2025-07-13', '27.7'], ['2025-07-14', '24.9'], ['2025-07-15', '21.6'], ['2025-07-16', '23.8'], ['2025-07-17', '25.8'], ['2025-07-18', '29.4'], ['2025-07-19', '22.5'], ['2025-07-20', '20.4'], ['2025-07-21', '22.6'], ['2025-07-22', '22.3'], ['2025-07-23', '22.0'], ['2025-07-24', '20.8'], ['2025-07-25', '25.4'], ['2025-07-26', '23.2'], ['2025-07-27', '21.5'], ['2025-07-28', '24.1'], ['2025-07-29', '21.9'], ['2025-07-30', '24.7'], ['2025-07-31', '22.6']]
[]

First 5 rows:
  2025-07-02: 23.0°C
  2025-07-03: 23.6°C
  2025-07-04: 26.1°C
  2025-07-05: 21.6°C
  2025-07-06: 22.5°C


**Alternative: Using DictReader**

The `csv.DictReader` gives you dictionaries instead of lists, which is often more convenient.

In [21]:
with open('data/july_2025_temps.csv', 'r') as f:
    reader = csv.DictReader(f)
    
    # Now each row is a dictionary
    rows = list(reader)
    print(rows)
    print("First 5 rows as dictionaries:")
    for row in rows[:5]:
        print(f"  {row['date']}: {row['max_temp_c']}°C")

[{'date': '2025-07-01', 'max_temp_c': '33.5'}, {'date': '2025-07-02', 'max_temp_c': '23.0'}, {'date': '2025-07-03', 'max_temp_c': '23.6'}, {'date': '2025-07-04', 'max_temp_c': '26.1'}, {'date': '2025-07-05', 'max_temp_c': '21.6'}, {'date': '2025-07-06', 'max_temp_c': '22.5'}, {'date': '2025-07-07', 'max_temp_c': '19.5'}, {'date': '2025-07-08', 'max_temp_c': '21.9'}, {'date': '2025-07-09', 'max_temp_c': '25.2'}, {'date': '2025-07-10', 'max_temp_c': '30.0'}, {'date': '2025-07-11', 'max_temp_c': '31.6'}, {'date': '2025-07-12', 'max_temp_c': '28.8'}, {'date': '2025-07-13', 'max_temp_c': '27.7'}, {'date': '2025-07-14', 'max_temp_c': '24.9'}, {'date': '2025-07-15', 'max_temp_c': '21.6'}, {'date': '2025-07-16', 'max_temp_c': '23.8'}, {'date': '2025-07-17', 'max_temp_c': '25.8'}, {'date': '2025-07-18', 'max_temp_c': '29.4'}, {'date': '2025-07-19', 'max_temp_c': '22.5'}, {'date': '2025-07-20', 'max_temp_c': '20.4'}, {'date': '2025-07-21', 'max_temp_c': '22.6'}, {'date': '2025-07-22', 'max_temp_

## Section 4: When to Use JSON vs CSV

**Use JSON when:**

- Your data has nested structure (dictionaries within dictionaries)
- You're storing API responses
- You need to preserve data types exactly

**Use CSV when:**

- Your data is simple rows and columns
- You want to open it in Excel or Google Sheets
- You're creating final summary tables

**In W04 Practice:** You'll collect data from an API (store as JSON for safety), then create a summary table (export as CSV for easy viewing).

## Practical Tips for Your W04 Work

1. **Always save raw API data immediately** - download once, save to JSON, work from the file
2. **Create a `data/` folder** - keep your data files organised
3. **Use descriptive filenames** - `london_temps_2004_2025.json` not `data1.json`
4. **Test with small samples first** - like we did with just July 2025 here

## Summary: Complete Workflow

Here's the pattern you'll use in W04 Practice:

```python
# 1. Collect data from API (run this ONCE)
response = requests.get(url, params=params)
data = response.json()

# 2. Save immediately to avoid re-downloading
with open('data/my_data.json', 'w') as f:
    json.dump(data, f, indent=2)

# 3. Read from file for all your analysis work
with open('data/my_data.json', 'r') as f:
    data = json.load(f)

# 4. Do your analysis with loops and conditionals
# (You'll learn this in Friday's lab)

# 5. Save results to CSV
with open('data/results.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['year', 'heatwave_count'])
    # ... write your results
```

---

**Next Steps:**

- Friday's lab will show you how to use if/else and for loops with this data
- W04 Practice will combine everything: API → JSON → loops/conditionals → CSV summary
- Keep this notebook as a reference for file operations